In [32]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit.circuit import QuantumCircuit
from copy import  deepcopy
from qiskit.circuit import Parameter,ParameterVector


# Define Molecule
molecule = MoleculeInfo(
    ["Li", "H"], [(0.0, 0.0, 0.0), (1.7, 0.0, 0.0)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)

problem = driver.run()
converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True)
transformer = FreezeCoreTransformer(
    freeze_core=True,  # how many electrons we have in our active space
    # how many orbitals we have in our active space
    remove_orbitals=[-3, -2]
)
problem = transformer.transform(problem=problem)

In [33]:
print(f'空间轨道数：{problem.num_spatial_orbitals}, α电子数：{problem.num_alpha}, β电子数：{problem.num_beta},\nα轨道占据情况：{problem.orbital_occupations}\
, β轨道占据情况：{problem.orbital_occupations_b}')
problem.properties

空间轨道数：5, α电子数：1, β电子数：1,
α轨道占据情况：[1. 0. 0. 0. 0.], β轨道占据情况：[1. 0. 0. 0. 0.]


In [34]:
print(problem.properties)

In [35]:
from qiskit_nature.second_q.circuit.library import UCCSD,UCC
ansatz_uccsd = UCCSD(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,qubit_converter=converter)
ansatz_uccgsd = UCCSD(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,qubit_converter=converter,generalized=True)
ansatz_uccgd = UCC(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,qubit_converter=converter,generalized=True,excitations='d')
print(f'uccsd 的激发列表个数为{len(ansatz_uccsd.excitation_list)},算符个数为{len(ansatz_uccsd.excitation_ops())}')
print(f'uccgsd 的激发列表个数为{len(ansatz_uccgsd.excitation_list)}')
print(f'uccgd 的激发列表个数为{len(ansatz_uccgd.excitation_list)}')
ansatz_uccsd.excitation_ops

uccsd 的激发列表个数为24,算符个数为24
uccgsd 的激发列表个数为130
uccgd 的激发列表个数为110


<bound method UCC.excitation_ops of <qiskit_nature.second_q.circuit.library.ansatzes.uccsd.UCCSD object at 0x7ff4d51b0520>>

In [28]:
ex_op = ansatz_uccsd.excitation_ops()
len(ex_op)

92